In [1]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
import sys
import utils
import base64
import pandas as pd

In [2]:
client = OpenAI()

In [3]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [35]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
        10: New roads, no defects.
        9: Very few defects (<10% surface issues).
        8: Minor defects (10-30% surface issues), no structural problems.
        7: Noticeable surface defects (>30%), no structural problems.
        6: Some cracking (<20%), good patching, minor distortions.
        5: Significant cracking (>20%), fair patching, notable distortions.
        4: Moderate structural issues (5-25%), frequent potholes.
        3: Major structural issues (25-50%), frequent potholes.
        2: Severe structural issues (>50%), many potholes.
        1: Complete road failure, extensive damage.
        """

In [36]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            },
                            {"type": "text", "text": "Focusing on the asphalt road, do you see any cracks, alligator cracking, potholes, ravelling, bleeding, rutting, surface distortion, edge breakup, patching, etc?"},
                            {"type": "text", "text":  road_grading_standards},
                            {"type": "text", "text": "Please provide a grade for the road condition based on the provided standards, only a single grade number. Please note that when identifying road anomalies, exclude the effects of shadows and water stains caused by light.Response me with only one word"},
                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [37]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model1_o_09211606.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)